In [1]:
# !pip install --upgrade --quiet  langchain langchain-community langchain_experimental langchain-groq neo4j

In [2]:
import dotenv
dotenv.load_dotenv()

True

In [3]:
import os
NEO4J_URI = os.environ["NEO4J_URI"]
NEO4J_USERNAME = os.environ["NEO4J_USERNAME"]
NEO4J_PASSWORD = os.environ["NEO4J_PASSWORD"]
GROQ_API_KEY = os.environ["GROQ_API_KEY"]


In [4]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [5]:
graph

In [6]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    groq_api_key=GROQ_API_KEY,
    model_name="llama-3.1-70b-versatile")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B9546BBA50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B952E22810>, model_name='llama-3.1-70b-versatile', groq_api_key=SecretStr('**********'))

In [7]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer=LLMGraphTransformer(llm=llm)

In [8]:
from langchain_core.documents import Document
text="""
Elon Reeve Musk (born June 28, 1971) is a businessman and investor known for his key roles in space
company SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,
formerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI.
He is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be
US$221 billion.Musk was born in Pretoria to Maye and engineer Errol Musk, and briefly attended
the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through
his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada.
Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics
 and physics. He moved to California in 1995 to attend Stanford University, but dropped out after
  two days and, with his brother Kimbal, co-founded online city guide software company Zip2.
 """
documents=[Document(page_content=text)]
documents

[Document(page_content="\nElon Reeve Musk (born June 28, 1971) is a businessman and investor known for his key roles in space\ncompany SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,\nformerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI.\nHe is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be\nUS$221 billion.Musk was born in Pretoria to Maye and engineer Errol Musk, and briefly attended\nthe University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through\nhis Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada.\nMusk later transferred to the University of Pennsylvania and received bachelor's degrees in economics\n and physics. He moved to California in 1995 to attend Stanford University, but dropped out after\n  two days and, with his brother Kimbal, co-founded online city guide 

In [9]:
graph_documents=llm_transformer.convert_to_graph_documents(documents)
graph_documents

[GraphDocument(nodes=[Node(id='Elon Musk', type='Person'), Node(id='Spacex', type='Company'), Node(id='Tesla, Inc.', type='Company'), Node(id='X Corp.', type='Company'), Node(id='The Boring Company', type='Company'), Node(id='Xai', type='Company'), Node(id='Neuralink', type='Company'), Node(id='Openai', type='Company'), Node(id='Maye Musk', type='Person'), Node(id='Errol Musk', type='Person'), Node(id='University Of Pretoria', type='University'), Node(id="Queen'S University", type='University'), Node(id='University Of Pennsylvania', type='University'), Node(id='Stanford University', type='University'), Node(id='Kimbal Musk', type='Person'), Node(id='Zip2', type='Company'), Node(id='Pretoria', type='City'), Node(id='Canada', type='Country'), Node(id='California', type='State')], relationships=[Relationship(source=Node(id='Elon Musk', type='Person'), target=Node(id='Spacex', type='Company'), type='FOUNDER'), Relationship(source=Node(id='Elon Musk', type='Person'), target=Node(id='Tesla, 

In [10]:
graph_documents[0].nodes

[Node(id='Elon Musk', type='Person'),
 Node(id='Spacex', type='Company'),
 Node(id='Tesla, Inc.', type='Company'),
 Node(id='X Corp.', type='Company'),
 Node(id='The Boring Company', type='Company'),
 Node(id='Xai', type='Company'),
 Node(id='Neuralink', type='Company'),
 Node(id='Openai', type='Company'),
 Node(id='Maye Musk', type='Person'),
 Node(id='Errol Musk', type='Person'),
 Node(id='University Of Pretoria', type='University'),
 Node(id="Queen'S University", type='University'),
 Node(id='University Of Pennsylvania', type='University'),
 Node(id='Stanford University', type='University'),
 Node(id='Kimbal Musk', type='Person'),
 Node(id='Zip2', type='Company'),
 Node(id='Pretoria', type='City'),
 Node(id='Canada', type='Country'),
 Node(id='California', type='State')]

In [11]:
graph_documents[0].relationships

[Relationship(source=Node(id='Elon Musk', type='Person'), target=Node(id='Spacex', type='Company'), type='FOUNDER'),
 Relationship(source=Node(id='Elon Musk', type='Person'), target=Node(id='Tesla, Inc.', type='Company'), type='FOUNDER'),
 Relationship(source=Node(id='Elon Musk', type='Person'), target=Node(id='X Corp.', type='Company'), type='OWNER'),
 Relationship(source=Node(id='Elon Musk', type='Person'), target=Node(id='The Boring Company', type='Company'), type='FOUNDER'),
 Relationship(source=Node(id='Elon Musk', type='Person'), target=Node(id='Xai', type='Company'), type='FOUNDER'),
 Relationship(source=Node(id='Elon Musk', type='Person'), target=Node(id='Neuralink', type='Company'), type='FOUNDER'),
 Relationship(source=Node(id='Elon Musk', type='Person'), target=Node(id='Openai', type='Company'), type='FOUNDER'),
 Relationship(source=Node(id='Elon Musk', type='Person'), target=Node(id='Maye Musk', type='Person'), type='CHILD_OF'),
 Relationship(source=Node(id='Elon Musk', typ

In [12]:
nodes = graph_documents[0].nodes

# Cicla su ogni nodo e costruisci una query dinamica per ogni tipo di nodo
for node in nodes:
    # Usa f-string per inserire dinamicamente il tipo di nodo
    node_query = f"""
    MERGE (n:`{node.type}` {{id: $id}})
    RETURN n
    """
    # Crea i parametri per il nodo corrente
    node_data = {'id': node.id}

    # Esegui la query per ogni nodo
    graph.query(node_query, params=node_data)

In [13]:
relationships = graph_documents[0].relationships

# Cicla su ogni relazione e costruisci una query dinamica per ogni tipo di relazione
for rel in relationships:
    # Usa f-string per inserire dinamicamente il tipo di relazione
    rel_query = f"""
    MATCH (a {{id: $source_node_id}})
    MATCH (b {{id: $target_node_id}})
    MERGE (a)-[r:`{rel.type}`]->(b)
    RETURN r
    """
    # Crea i parametri per la relazione corrente
    rel_data = {
        'source_node_id': rel.source.id,
        'target_node_id': rel.target.id
    }

    # Esegui la query per ogni relazione
    graph.query(rel_query, params=rel_data)

In [14]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Company {id: STRING}
Person {id: STRING}
University {id: STRING}
City {id: STRING}
Country {id: STRING}
State {id: STRING}
Relationship properties:

The relationships:
(:Person)-[:FOUNDER]->(:Company)
(:Person)-[:CHILD_OF]->(:Person)
(:Person)-[:OWNER]->(:Company)
(:Person)-[:ATTENDED]->(:University)
(:Person)-[:CO-FOUNDER]->(:Person)
(:Person)-[:CO-FOUNDER]->(:Company)
(:Person)-[:BORN_IN]->(:City)
(:Person)-[:IMMIGRATED_TO]->(:Country)
(:Person)-[:MOVED_TO]->(:State)


In [15]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x000001B9400F1A90>, cypher_generation_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'schema'], template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B9546BBA50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B952E22810>, model_name='llama-3.1-70b-versatile', groq_api_key=SecretStr('**

In [16]:
response=chain.invoke({"query":"Elon Musk is the founder of"})

response




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (person:Person)-[:FOUNDER]->(company:Company) WHERE person.id = "Elon Musk" RETURN company.id
Full Context:
[{'company.id': 'Spacex'}, {'company.id': 'Tesla, Inc.'}, {'company.id': 'The Boring Company'}, {'company.id': 'Xai'}, {'company.id': 'Neuralink'}, {'company.id': 'Openai'}]

> Finished chain.


{'query': 'Elon Musk is the founder of',
 'result': 'Elon Musk is the founder of Spacex, Tesla, Inc., The Boring Company, Xai, Neuralink, Openai.'}